# Laboratorium 3: Wprowadzenie do biblioteki Python IBM GenAI

Witamy w laboratorium 3.

W poprzednim laboratorium badaliśmy wyzwania związane z prompt engineeringiem; ucząc się, jak dostosować nasze sformułowania, wybrać inny model i zoptymalizować parametry modelu. Niewielkie zmiany mogą znacznie poprawić wyniki generowane przez modele językowe

W tym laboratorium zastosujemy naszą nową wiedzę w rzeczywistym przypadku użycia, przechodząc od prompt engineeringu w watsonx.ai do prompt engineeringu w Pythonie. Korzystając z [biblioteki IBM GenAI Python](https://ibm.github.io/ibm-generative-ai/) do programowej interakcji z watsonx.ai, użyjemy szablonów, aby usprawnić naszą interakcję z modelem językowym i zmaksymalizować jego potencjał.

Koncepcja Prompt Patterns zapewniana przez bibliotekę GenAI Python firmy IBM umożliwia konstruowanie szablonów, które można łatwo wypełnić określonymi informacjami w celu wygenerowania szerokiego zakresu danych wyjściowych.

# Uwaga
**Biblioteka IBM GenAI Python jest obecnie w wersji Beta i ulegnie zmianie w przyszłości.**

Biblioteka `ibm-generative-ai` wymaga `python>=3.9`.

## Przetworzenie danych wejściowych prompt engineeringu do promptów używanych przez bibliotekę GenAI o nazwie Prompt Patterns

### Scenariusz: spersonalizowana rekomendacja dla firmy detalicznej XYZ <a id="step3"></a>

XYZ Retail to popularny internetowy sklep detaliczny, który sprzedaje szeroką gamę produktów, w tym elektronikę, odzież, artykuły gospodarstwa domowego i wiele innych. Mają dużą bazę klientów i chcą zapewnić spersonalizowane zakupy, aby zwiększyć satysfakcję klientów i zwiększyć sprzedaż.

Aby osiągnąć ten cel, XYZ chce wykorzystać generatywną sztuczną inteligencję do tworzenia arkuszy informacyjnych na temat każdego ze swoich klientów. Te arkusze informacyjne zawierają podsumowanie istotnych informacji, takich jak dane demograficzne klientów (imię i nazwisko, wiek, lokalizacja) oraz historia zakupów. Te arkusze informacyjne pomogą zespołowi sprzedaży XYZ Retail budować silniejsze relacje z klientami, zwiększać zadowolenie klientów i zachęcać do powtarzania zakupów.


Rozpoczynasz od wykonania przekazania danych wejściowych w narzędziu Prompt Lab, a natępnie możesz przetestować dane wyjściowe modelu podstawowego za pomocą pdanych wejściowych, takich jak tu:

![title](images/prompt_without_example.png)

Rekomendacja modelki nie jest dokładna ani użyteczna, ponieważ klient Michael Jones kupił zabawki i gry, a nie odzież sportową na świeżym powietrzu. Na szczęście w laboratorium Prompt Engineering dowiedziałeś się, że nauka kilku strzałów może pomóc w uzyskaniu lepszych wyników.

Co się stanie, gdy podamy kilka przykładów za pomocą Prompt Builder, aby poprowadzić LLM do generowania bardziej znaczących rekomendacji.


![title](images/prompt_with_example.png)


Świetnie, rekomendacja produktu dla Michaela Jonesa jest znacznie lepsza. Jak jednak wyprodukować kilka zachęt do generowania rekomendacji dla wszystkich klientów detalicznych XYZ? Kopiowanie i wklejanie informacji każdego klienta do narzędzia Prompt Builder zajęłoby zbyt dużo czasu.

Będziesz potrzebować rozwiązania programistycznego. Być może mógłbyś nawet wygenerować duży zestaw przykładów, a następnie użyć go do dostrajania modelu w Watsonx.ai. 

Ale wyprzedzamy samych siebie, ponieważ dowiesz się o tworzeniu zestawu danych Prompt Tuning w późniejszym laboratorium.


<p align="center">
  <img src="images/scenario_flow_chart03.png" width="600"/>
</p>

## 1. Load the required libraries  <a id="step1"></a>

In [1]:
import os
from genai.model import Credentials, Model
from genai.schemas import GenerateParams
from genai.prompt_pattern import PromptPattern
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
import json 

/Users/joannafrankiewicz/anaconda3/lib/python3.10/site-packages/genai/schemas/__init__.py:10: DeprecationWarning: 
The class ModelType is being deprecated.
Please replace any reference to ModelType by its model id string equivalent.
Example :
  ModelType.FLAN_T5 becomes "google/flan-t5-xxl"

  from genai.schemas.models import ModelType


## 2. Utwórz arkusz informacyjny dla każdego klienta, przy użyciu Prompt Patterns  <a id="step2"></a>

### **2.1 What are Prompt Patterns?**

[Klasa PromptPattern](https://ibm.github.io/ibm-generative-ai/rst_source/genai.prompt.prompt_pattern.html) w [Bibliotece IBM GenAI](https://ibm.github.io/ibm-generative-ai/) zapewnia elastyczne podejście do tworzenia danych wejściowych z ustrukturyzowanych szablonów. Użyjemy klasy PromptPattern, aby uprościć tworzenie naszych kilku podpowiedzi dla XYZ Retail.


Firma XYZ Retail udostępniła Ci dane swoich klientów w formacie .csv. Aby wygenerować podpowiedzi dla każdego klienta, należy przekształcić podpowiedzi opracowane w narzędziu Prompt Builder do bardziej przydatnego formatu programistycznego. Używając klasy Prompts Pattern, możesz łatwo zastąpić dane klienta z pliku, aby wygenerować jeden lub wiele danych wejściowych.

Klasa PromptPattern definiuje schemat, w którym zmienne do zastąpienia są umieszczane w podwójnych nawiasach klamrowych „{{}}”. Te nawiasy klamrowe służą jako element zastępczy dla rzeczywistych danych, które zostaną zastąpione w szablonie.

Zobaczmy, jak to działa w praktyce.

### **2.2 Tworzenie prostych danych wejściowych z szablonu**

Wzorzec zachęty można utworzyć za pomocą klasy PromptPattern z ciągu znaków, pliku lub adresu URL. Tutaj są [dodatkowe przykłady PromptPattern](https://ibm.github.io/ibm-generative-ai/rst_source/examples/examples.html), które uwzględniono w dokumentacji.

#### 2.2.1 Prompt Pattern z danych tekstowych

In [2]:
pattern = "input: {{name}} {{family_name}} is {{age}} and lives in {{location}}. They bought {{purchase_history}}"

prompt = PromptPattern.from_str(pattern)
prompt.sub("name", "Jane").sub("family_name", "Doe").sub("age", "43").sub("location", "San Francisco, CA").sub("purchase_history", "groceries, household goods and travel supplies")

prompt

input: Jane Doe is 43 and lives in San Francisco, CA. They bought groceries, household goods and travel supplies

#### 2.2.2 Prompt Pattern z pliku
Prompt patterns mogą być również przechowywane w pliku .yaml

In [3]:
_path_to_file = "../templates/customer_factsheet.yaml"

prompt = PromptPattern.from_file(_path_to_file)
print("TEMPLATE:\n" + str(prompt))

# This template can now be populated by iterating over an array:
names= ["Jane", "Siamak", "Luis"]
family_names = ["Doe", "Baharoo", "Cooli"]
ages = [43, 57, 21]
cities = ["San Francisco", "Chicago", "New York City"]
states= ["CA", "IL", "NY"]
purchase_histories= ["groceries, household goods and travel supplies", "Books electronics home_goods", "Clothing shoes cosmetics"]
recommendation_1s= ["Basket of organic fruits", 
                    "Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.", 
                    "Aritzia Wilfred Free Sweater - This soft and cozy sweater is perfect for a casual day out. It's available in a variety of colors, so you can find the perfect one to match your style."]
recommendation_2s= ["Lightweight carry-on suitcase", 
                    "Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family.", 
                    "Steve Madden Pointed Toe Pumps - These stylish pumps are perfect for a night out on the town. They're comfortable and versatile, so you can wear them with a variety of outfits."]
for x in range(1, 4):
    prompt.sub(f"name_{x}", names[x-1]).sub(f"family_name_{x}", family_names[x-1]).sub(f"age_{x}", str(ages[x-1])).sub(f"city_{x}", cities[x-1]).sub(f"state_{x}", states[x-1])
    prompt.sub(f"purchase_history_{x}", purchase_histories[x-1])
    prompt.sub(f"recommendation_1_{x}", recommendation_1s[x-1])
    prompt.sub(f"recommendation_2_{x}", recommendation_2s[x-1])
print("\nPOPULATED TEMPLATE:\n" + str(prompt))

TEMPLATE:
input: "{{name_1}} {{family_name_1}} is {{age_1}} years old and lives in {{city_1}}, {{state_1}}. Their purchase history includes {{purchase_history_1}}."
output: "Recommendations:\n Item 1: {{recommendation_1_1}}\nItem 2: {{recommendation_2_1}}"
input: "{{name_2}} {{family_name_2}} is {{age_2}} years old and lives in {{city_2}}, {{state_2}}. Their purchase history includes {{purchase_history_2}}."
output: "Recommendations:\n Item 1: {{recommendation_1_2}}\nItem 2: {{recommendation_2_2}}"
input: "{{name_3}} {{family_name_3}} is {{age_3}} years old and lives in {{city_3}}, {{state_3}}. Their purchase history includes {{purchase_history_3}}."
output: ""


POPULATED TEMPLATE:
input: "Jane Doe is 43 years old and lives in San Francisco, CA. Their purchase history includes groceries, household goods and travel supplies."
output: "Recommendations:\n Item 1: Basket of organic fruits\nItem 2: Lightweight carry-on suitcase"
input: "Siamak Baharoo is 57 years old and lives in Chicago, 

## 3. Stwórz Prompty bazując na zestawieniu informacji o klientach <a id="step3"></a>


### 3.1 Masowe tworzenie danych wejściowych
Możemy teraz wygenerować "few shot prompts" z wierszy w pliku csv za pomocą „sub_all_from_csv”. Można to również zrobić z json. Szczegóły można znaleźć w [dokumentacji klasy PromptPattern](https://ibm.github.io/ibm-generative-ai/rst_source/genai.prompt.prompt_pattern.html)

In [4]:
_path_to_template_file = "../templates/customer_factsheet.yaml"
_path_to_csv_file = "../data/customer_factsheet.csv"

prompt = PromptPattern.from_file(_path_to_template_file)
print("TEMPLATE:\n" + str(prompt))

# We use a mapping table like below so each row in the CSV is mapped to the correct variable of the YAML template.
# The recommendations for rows 1 and 2 are used for few shot training while every row's 3rd recommendation
# is ignored as these are the recommendation for which we are evaluating our prompt's performance.

# Używamy tabeli mapowania, jak poniżej, więc każdy wiersz w pliku CSV jest mapowany na odpowiednią zmienną szablonu YAML.
# Zalecenia dla wierszy 1 i 2 są używane do "few shot training", podczas gdy 3. zalecenie dla każdego wiersza
# jest ignorowane, ponieważ są to zalecenia, dla których oceniamy wydajność naszych danych wejściowych.
mapping = {
    "name": ["name_1", "name_2", "name_3"],
    "family_name": ["family_name_1", "family_name_2", "family_name_3"],
    "age": ["age_1", "age_2", "age_3"],
    "city": ["city_1", "city_2", "city_3"],
    "state": ["state_1", "state_2", "state_3"],
    "purchase_history": ["purchase_history_1", "purchase_history_2", "purchase_history_3"],
    "recommendation_1": ["recommendation_1_1", "recommendation_1_2", "recommendation_1_3"],
    "recommendation_2": ["recommendation_2_1", "recommendation_2_2", "recommendation_2_3"]
}

list_of_prompts = prompt.sub_all_from_csv(csv_path=_path_to_csv_file,col_to_var=mapping)

print("\nPOPULATED TEMPLATE:\n" + str(list_of_prompts[0]))

TEMPLATE:
input: "{{name_1}} {{family_name_1}} is {{age_1}} years old and lives in {{city_1}}, {{state_1}}. Their purchase history includes {{purchase_history_1}}."
output: "Recommendations:\n Item 1: {{recommendation_1_1}}\nItem 2: {{recommendation_2_1}}"
input: "{{name_2}} {{family_name_2}} is {{age_2}} years old and lives in {{city_2}}, {{state_2}}. Their purchase history includes {{purchase_history_2}}."
output: "Recommendations:\n Item 1: {{recommendation_1_2}}\nItem 2: {{recommendation_2_2}}"
input: "{{name_3}} {{family_name_3}} is {{age_3}} years old and lives in {{city_3}}, {{state_3}}. Their purchase history includes {{purchase_history_3}}."
output: ""


POPULATED TEMPLATE:
input: "John Smith is 30 years old and lives in San Francisco, CA. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It h

### 3.2 Dodatkowe przykłady
Możesz sprawdzić [dodatkowe przykłady użycia Prompt Pattern](https://ibm.github.io/ibm-generative-ai/rst_source/examples/prompts.html)

## 4. Szybka ocena i "few shot learning" na podstawie danych wejściowych utworzonych zbiorczo<a id="step4"></a>

W poprzednich przykładach utworzyłeś dane wejściowe za pomocą „2-shot learning”. Tj. były trzy wejścia, ale tylko dwa kompletne wyjścia. Używając w ten sposób większego zestawu danych, możesz przeprowadzić masowe testowanie promptu.

Np. dwie z twoich próbek danych są używane do trenowania, podczas gdy „wyjście” trzeciej próbki można porównać z danymi wyjściowymi modelu, aby upewnić się, że prompt działa zgodnie z oczekiwaniami. Możesz teraz wykonać "few shot prompts, aby zobaczyć, jak dobrze nasz opracowany prompt działa w licznych przykładach

### 4.1 Załaduj swoje dane logowania do watsonx.ai i wywołaj model

In [19]:
#Credentials
my_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : ""
}
project_id = ""
model_id    = ModelTypes.FLAN_T5_XXL
gen_parms   = None
space_id    = None
verify      = False
model = Model( model_id, my_credentials, gen_parms, project_id, space_id, verify )   

### 4.2 Send prompts to Watsonx.ai

In [79]:
for prompt in list_of_prompts:
    prompt_txt = str(prompt)
    responses = model.generate(prompt_txt)
    result = responses["results"]
    print(f"Prompt: {prompt}")
    print(f"Output: {result}")
    print("\n\n\n")

Prompt: input: "John Smith is 30 years old and lives in San Francisco, CA. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.\nItem 2: Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family."
input: "Jane Doe is 25 years old and lives in New York, NY. Their purchase history includes Clothing shoes cosmetics."
output: "Recommendations:\n Item 1: Aritzia Wilfred Free Sweater - This soft and cozy sweater is perfect for a casual day out. It's available in a variety of colors, so you can find the 

### Few shot prompt analiza
Te wyniki nie są złe. X-Box dla klienta z historią kupowania zabawek i gier. Podobnie kosmetyki i meble dla pozostałych dwóch klientów dokładnie odzwierciedlają ich historię zakupów.

## 5. Gratulacje
Gratulujemy ukończenia laboratorium i odkrywania fascynującego świata masowego tworzenia podpowiedzi Few Shot Prompts przy użyciu PromptPatterns!

Dzięki praktycznemu przykładowi generowania spersonalizowanych rekomendacji produktów przekonałeś się o sile dostosowywania danych wejściowych do indywidualnych profili klientów. Uwzględniając szczegóły specyficzne dla klienta i programowo generując zbiorcze przykłady, możesz dostosować model do konkretnego przypadku użycia, co skutkuje dokładniejszymi i dostosowanymi wynikami. 

Continuously iterating and refining your prompts based on these examples will unlock the full potential of language models and enhance their performance across various domains. Keep experimenting and leveraging prompt engineering techniques to optimize your interactions with language models and drive impactful results in your projects.